In [17]:
import numpy as np
import torch
import Model
from torchvision import datasets, transforms
import Args
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
import pandas
import threading
import multiprocessing
import data.ff
from HCFedAvg.DataGenerater import *


In [18]:
from torch.utils.data import TensorDataset
from torchvision.utils import save_image
args = Args.Arguments()
DatasetGen(args)


# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
# for data, label in train_loader:
#     print(data.shape)


0     804
1     806
2     805
3     807
4     804
tensor([[[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
  

In [10]:
import Model

model1 = Model.init_model('mnist')
len(model1.fc4.weight[0])

20

In [13]:
for i in range(1, 5):
    print(i)

1
2
3
4


In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

train_dataset = datasets.MNIST(root='data', train=True, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))]))


In [10]:
import Data
from KMeansPP import get_cos_dis_single_layer
from ClusterMain import pca_dim_deduction
#
list1 = torch.load('DeepModelSimality/param_grad_1_.pt')
list2 = torch.load('DeepModelSimality/param_grad_2_.pt')
#
# list1 = torch.load('DeepModelSimality/param1.pt')
# list2 = torch.load('DeepModelSimality/param2.pt')
new_liat1 = [np.array(l.tolist()[0]) for l in list1]
new_liat2 = [np.array(l.tolist()[0]) for l in list2]

#
low_dim_list1 = pca_dim_deduction(np.array(new_liat1), 3)
low_dim_list2 = pca_dim_deduction(np.array(new_liat2), 3)
# torch.tensor(new_liat1[99])
#
# for i, value in enumerate(list1):
#     dis = torch.pairwise_distance(list1[i][0].cpu(), list1[0][0].cpu(), p=2)
#     print(dis)

s_list = []
cos_dis = []
for i in range(len(new_liat1)):
    s_list.append(torch.pairwise_distance(torch.tensor(new_liat1[i]) , torch.tensor(new_liat2[0]), p = 2).tolist())
    cos_dis.append(get_cos_dis_single_layer(torch.tensor(new_liat1[i]) , torch.tensor(new_liat2[0])).tolist())
print(s_list)

print("    cos    ")
print(cos_dis)

[0.00762958332698205, 0.0074733093958950945, 0.08613762119662634, 0.007877143544110036, 0.007845166654261167, 0.007875629390036953, 0.009684006805919755, 0.007880705998888756, 0.007682926863103191, 0.007866505068415759, 0.007869765326197658, 0.007636403581663664, 0.007883628872696394, 0.007883054593261456, 0.00788229339918589, 0.007879171482255663, 0.007882268920787824, 0.007885433690536393, 0.007852880180128197, 0.007882437393605153, 0.008001766790038625, 0.007882268756868129, 0.0078823158352659, 0.007882629776385656, 0.007881784503957266, 0.007881859924934438, 0.007882272771686407, 0.00788216761415598, 0.00836890026030093, 0.007878263444534517, 0.007882275258683123, 0.0077768148010795585, 0.007882252389401122, 0.007883090392497133, 0.007882275812228378, 0.00788285225005749, 0.00788227569677405, 0.007884071441357261, 0.012886741904281511, 0.00788227577776396, 0.007882275852418137, 0.007882293048572896, 0.007882254616280318, 0.00788221701711699, 0.007882308494058781, 0.0078822757923457

In [20]:

new_dict = torch.load(
               "DeepModelSimality" + '/' + 'new.pt')
new_dict

{0: [tensor([0., 0., 0., 0., 0.])]}